In [29]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob

In [30]:
tf.test.is_built_with_cuda()

True

In [31]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [32]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [34]:
building1_filenames = set(glob.glob("datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("datasets/building1/train/*.feather"))
test1_filenames = building1_filenames - training_filenames

In [35]:
import random
seed=10
random.seed(seed)
np.random.seed(seed)

In [36]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)

#training_data.head()

orientation_acc = training_data[["iphoneAccX", "iphoneAccY", "iphoneAccZ"]]
orientation_gyro = training_data[[ "iphoneGyroX", "iphoneGyroY", "iphoneGyroZ"]]
orientation_mag = training_data[[ "iphoneMagX", "iphoneMagY", "iphoneMagZ"]] 

#orientation_data.head()

In [37]:
orientation_target = training_data[["orientX", "orientY", "orientZ", "orientW"]] 
orientation_target.head()

,orientX,orientY,orientZ,orientW
0,0.006522,0.002056,-0.000054,0.999977
1,0.006549,0.002054,-0.000135,0.999976
2,0.006513,0.002045,-0.000189,0.999977
3,0.006550,0.002060,-0.000221,0.999976
4,0.006574,0.002041,-0.000326,0.999976


In [38]:
checkpoints_path = os.path.join(os.getcwd(),'datasets/CheckPoints_Model_OrientNet')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_vloss_{val_loss:.4f}.hdf5')
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [39]:
orient_learning_rate = 0.0005 # convergence  within  20  epochs \ref{IDOL}

In [40]:
# Loss Function
class OreintLoss(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # custom loss L_orient = 0.5*(q [-] \hat{q})^T *Sig^{-1}* 
    W = 25
    ws = tf.constant([W,W,1,1,1,1],dtype=tf.float32)
    return tf.sqrt(tf.reduce_mean(ws*(y_pred - y_true)**2, axis=-1))

In [41]:
class OreintNet():
    def build_model(self):
        # Building Network
        #1. Define inputs
        Acc_input = Input(shape=(1,3) , name = 'Acc_input')    
        Gyro_input = Input(shape=(1,3) , name = 'Gyro_input')
        Mag_input = Input(shape=(1,3) , name = 'Mag_input')
        
        MergedLayer = Concatenate()([Acc_input , Gyro_input, Mag_input])
        #
        ##MergedLayer = Input(shape=(9) , name = 'imu_input')
        #2. LSTM
        LSTM1 = (LSTM(100 , return_sequences=True))(MergedLayer) # , return_sequences=True
        LSTM2 = (LSTM(100))(LSTM1)
        
        #3. Fully-Connected (Sigma)
        Dense1 = Dense(units=100, activation='tanh')(LSTM2)
        Dense2 = Dense(units=16, activation='tanh')(Dense1)
        sigma_output = Dense(units=6, activation='linear')(Dense2) # Sigma Outputs
        
        #4. Fully-Connected (Theta)
        Dense3 = Dense(units=100, activation='tanh')(LSTM2)
        Dense4 = Dense(units=16, activation='tanh')(Dense3)
        theta_output = Dense(units=6, activation='linear')(Dense4) # Theta Outputs
        
        #5. Define and compile The model
        Network = Model([MergedLayer] ,  [theta_output]) # , sigma_output])  #  [Acc_input,Gyro_input, Mag_input]
        Network.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate) ,metrics=['mape']) #loss=OreintLoss()
        return Network
        

In [42]:
network_builder = OreintNet()
orientation_network = network_builder.build_model()

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: concatenate_2/concat:0


ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 1, 3), dtype=tf.float32, name='Acc_input'), name='Acc_input', description="created by layer 'Acc_input'") at layer "concatenate_2". The following previous layers were accessed without issue: []

In [ ]:
orientation_network.summary()

In [ ]:
Nepochs = 20
history = orientation_network.fit(
    orientation_data,
    orientation_target,
    epochs=Nepochs,
    initial_epoch=initial_epoch,
    batch_size=64,
    callbacks = [check_point_callback]
)

In [ ]:
pos_learning_rate = 0.001